In this section, we will implement the Google PageRank algorithm using two different approaches. First, we will view it as a Markov process, simulating the random walk behavior of a user moving between webpages. Then, we will take a more deterministic approach by solving the problem using matrix factorization, which involves setting up and solving a system of linear equations.

We will also explain why both methods ultimately produce the same ranking after convergence.

Before diving into the PageRank computation, we will generate a graph represented by an adjacency matrix for a directed graph. In this graph, each outgoing link from a node represents a hyperlink from the current webpage to another webpage, and each incoming edge to a node represents a hyperlink from another webpage to the current one.

In [8]:
''' import libraries '''
import networkx as nx
import numpy as np
import plotly.graph_objects as go

In [9]:
''' generate adjacency matrix '''

def generateADJ(n = 10, density = 0.3):
    """ 
    Generate an n×n adjacency matrix where the 
    ratio of ones to n^2 is approximately equal
    to the specified density parameter.
    """
    random_matrix = np.random.rand(n, n)  
    adj_matrix = (random_matrix < density).astype(int)  
    
    # Make the matrix symmetric for undirected graphs (optional)  
    adj_matrix = np.triu(adj_matrix, 1)  # Keep only upper triangle  
    adj_matrix += adj_matrix.T  # Make it symmetric  
    
    return adj_matrix 
    
adj = generateADJ()
adj

array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 1, 0, 1, 0, 0, 1, 0, 1],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 1],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 1, 1, 0, 1, 0, 1, 1, 0]])

In [10]:
''' construct the graph using nx '''

def create_graph(adj):
    """ 
    Constructs a directed graph from the given adjacency matrix.
    """

    # Create a directed graph  
    G = nx.DiGraph()  
    
    # Iterate over the adjacency matrix and add edges  
    n = adj.shape[0]  # Number of nodes  
    for i in range(n):  
        for j in range(n):  
            if adj[i, j] == 1:  # If there's an edge from node i to node j  
                G.add_edge(i, j)  # Add directed edge from i to j  
    
    return G 

graph = create_graph(adj)
graph

In [11]:
''' visualize your generated graph '''

"""
Just run this cell.
Whenever you want to visualize the graph based on 
your current values, activate the test mine flag.
"""


def visualize_graph(G, testMine = False , myProbs = []):
    pos = nx.spring_layout(G, seed=42) 
    
    edge_x = []
    edge_y = []
    arrow_x = []
    arrow_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

        arrow_x.append((x0 + x1) / 2)  
        arrow_y.append((y0 + y1) / 2)
    
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=1, color='black'),
        hoverinfo='none',
        mode='lines'
    )

    arrow_trace = go.Scatter(
        x=arrow_x, y=arrow_y,
        mode='markers',
        marker=dict(
            size=8,
            color='black',
            symbol='triangle-up',  
            angle=-90  
        ),
        hoverinfo='none'
    )

    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale='YlGnBu', 
            size=20,  
            color=[],
            colorbar=dict(
                thickness=15,
                title='PageRank Value',
                xanchor='left',
                titleside='right'
            ),
            line_width=2))

    node_color = []
    node_text = []
    pagerank_values = nx.pagerank(G)  
    for node in G.nodes():
        if testMine:
            node_color.append(myProbs[node])
            node_text.append(f'Page {node} - PageRank: {myProbs[node]:.3f}')
        else:
            node_color.append(pagerank_values[node])  
            node_text.append(f'Page {node} - PageRank: {pagerank_values[node]:.3f}')
    
    node_trace.marker.color = node_color
    node_trace.text = node_text

    fig = go.Figure(data=[edge_trace, arrow_trace,  node_trace],
                    layout=go.Layout(
                        title='Webpage Linkage Graph (PageRank Visualization)',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=40),
                        annotations=[dict(
                            text="Directed graph representing webpage linkages",
                            showarrow=False,
                            xref="paper", yref="paper",
                            x=0.005, y=-0.002)],
                        xaxis=dict(showgrid=False, zeroline=False),
                        yaxis=dict(showgrid=False, zeroline=False)))

    fig.show()
    return pagerank_values

goalProbs = visualize_graph(graph)
print(f'Actual pagerank values : {goalProbs}')
myProbs = {key : 1/len(goalProbs) for key in goalProbs}
visualize_graph(graph, testMine=True , myProbs=myProbs)
print(f'Values I found : {myProbs}')


Actual pagerank values : {0: 0.049600466709609034, 8: 0.12211804138958529, 1: 0.04841626473069932, 5: 0.1965643871517563, 2: 0.080704727070993, 9: 0.18993354337728946, 3: 0.09399337594284683, 6: 0.05494773511591966, 4: 0.04841626473069932, 7: 0.11530519378060204}


Values I found : {0: 0.1, 8: 0.1, 1: 0.1, 5: 0.1, 2: 0.1, 9: 0.1, 3: 0.1, 6: 0.1, 4: 0.1, 7: 0.1}


Repeat the 3 cells above as many times as necessary until you reach a graph you are happy with. Keep in mind, as you have seen, the plot is interactive, allowing you to see the names of the nodes and their assigned values. Additionally, there are two extra parameters passed to the function: 

1. The first parameter is for testing mode, which lets you view the ranking based on your current state values. If not in testing mode, the default is the true ranking value.
   
2. The second parameter is the dictionary you have prepared for your test.

From here, we will explore a Markovian process to determine the stationary ranking we can reach by setting the convergence threshold. We start from uniformly distributed probabilities.


In the transposed version of the adjacency matrix, column *i* indicates to which other nodes we can proceed. By normalizing the columns so that each sums up to one, we can observe the probability of transitioning from node *i* to node *j*. This means that, to determine the probability of moving from node *i* to node *j*, we look at column *i* and row *j*.

When this matrix is multiplied by the current distribution of probabilities (the probability of being at each node, stored in `myProbs`), we get new probabilities. These represent the updated distribution after one iteration, according to the equation:

$
P_{t+1} = T \cdot P_t
$

Here, we implement this process over a certain number of iterations and plot the difference between the current probability vector and the previous one at each iteration. This allows us to track the convergence and understand when the distribution stabilizes.


In [12]:
''' Simulate the random walk '''

# def find_stationary_values(transitionMatrix, currentValues, iterations = 150, treshold = 1e-15):
#     """
#     Parameters:
# 
#     - transitionMatrix : adj matrix which you need to normalize.
#     - currentValues : initial probabilities which we initialized with a unifirm distribution.
#     - iterations : The maximum number of iterations to perform the random walk.
#     - treshold: The threshold for the difference ( you can use L1 norm ) between consecutive iterations. 
#                 The simulation stops when the difference falls below this value.
#     """
#     def normalize_transition_matrix(transitionMatrix):  
#         """ Normalize the adjacency matrix to create a transition matrix. """  
#         # Convert to float to avoid integer division  
#         transition_matrix = adj.astype(float)  
#         column_sums = transition_matrix.sum(axis=0)  
#         # Avoid division by zero by replacing zeros with ones  
#         column_sums[column_sums == 0] = 1  
#         normalized = transition_matrix / column_sums  
#         return normalized  
#     
#     normalizedTransitions = normalize_transition_matrix(transitionMatrix)  
#     myProbs = np.array(currentValues, dtype=float)  
#     
#     # Store the differences for visualization  
#     differences = []  
# 
#     for _ in range(iterations):  
#         newProbs = normalizedTransitions @ myProbs  # Matrix multiplication  
#         
#         # Calculate the L1 norm (sum of absolute differences)  
#         difference = np.sum(np.abs(newProbs - myProbs))  
#         differences.append(difference)  
#         
#         # Check for convergence  
#         if difference < treshold:  
#             break  
#         
#         myProbs = newProbs  # Update probabilities for the next iteration  

    # Visualization of the convergence    

def normalize_transition_matrix(adj):  
    """ Normalize the adjacency matrix to create a transition matrix. """  
    transition_matrix = adj.astype(float)  # Convert to float  
    column_sums = transition_matrix.sum(axis=0)  
    
    for i in range(transition_matrix.shape[1]):  
        if column_sums[i] == 0:  # Check for dangling nodes  
            transition_matrix[:, i] = 1.0 / transition_matrix.shape[0]  # Uniform distribution  
        else:  
            transition_matrix[:, i] /= column_sums[i]  # Normalize the column 
    return transition_matrix  

def find_stationary_values(transitionMatrix, currentValues, iterations=150, threshold=1e-15):  
    """  
    Simulate the random walk and find stationary values.  
    
    Parameters:  
    - transitionMatrix: The normalized adjacency matrix.  
    - currentValues: Initial probabilities (uniform distribution).  
    - iterations: The maximum number of iterations to perform.  
    - threshold: The threshold for the difference between consecutive iterations.  
    
    Returns:  
    - normalizedTransitions: The final normalized transition matrix.  
    """  
    normalizedTransitions = normalize_transition_matrix(transitionMatrix)  
    myProbs = np.array(currentValues, dtype=float)  
    differences = []  

    for _ in range(iterations):  
        newProbs = normalizedTransitions @ myProbs  # Matrix multiplication  
        difference = np.sum(np.abs(newProbs - myProbs))  # L1 norm  
        differences.append(difference)  
        
        if difference < threshold:  
            break  
        
        myProbs = newProbs  # Update probabilities  

    # Plotting with Plotly  
    fig = go.Figure()  
    fig.add_trace(go.Scatter(  
        x=list(range(len(differences))),  
        y=differences,  
        mode='lines+markers',  
        name='Difference (L1 norm)',  
        line=dict(shape='linear'),  
        marker=dict(size=6)  
    ))  
    
    # Update layout for log scale  
    fig.update_layout(  
        title='Convergence of Probability Distribution',  
        xaxis_title='Iteration',  
        yaxis_title='Difference (L1 norm)',  
        template='plotly_white'  
    )  
    
    fig.show()
    # Return the normalized transition matrix for reference  
    visualize_graph(graph, testMine=True, myProbs=myProbs)
    visualize_graph(graph)

    return normalizedTransitions


normalizedTransitions = find_stationary_values(adj, list(myProbs.values()))

You might encounter some dangling nodes in the graph, which are nodes with no outgoing edges. When the random walker reaches such a node, no other node is reachable from it, potentially causing `NaN` values in the normalized transition matrix. To handle this, you can assume that the random walker can move to any other node with equal probability. 

In the context of the transition matrix, this means that you should adjust the matrix to account for these dangling nodes. Specifically, for dangling nodes, replace the corresponding columns with a uniform distribution, ensuring that each node has an equal chance of being the next node.


## Using decomposition 

Now that we have achieved a ranking that closely matches the actual values, we can proceed with a new approach. 

When we reach the stable distribution, the transition matrix $ T $ will no longer affect the current state. In other words, we are looking for a vector $ p $ such that:

$ p_t = T p_t $

Solving for $ p $ is equivalent to finding the eigenvector corresponding to the eigenvalue 1 of $ T $. To do this, we will use QR decomposition to solve the equation. 

### Tasks

1. **Implement an Efficient System of Equation Solver**:
   - Use QR decomposition to solve $ (T - I)p = 0 $, where $ I $ is the identity matrix.
   - - To find the null space of $ T - I $, we use the Gram-Schmidt process on $ (T - I)^T $ to obtain an orthonormal basis for the row space of $ T - I $. When examining vectors corresponding to larger diagonal values of $ R $, we can determine that these vectors do contribute significantly to the basis of the row space, while the remaining vectors form the null space. This method provides an approximate solution that is close to the exact answer. However, be aware that dividing by the norm in the Gram-Schmidt process can lead to numerical instability, so increasing the precision of calculations is advisable.

2. **Compare Results**:
   - Compare the computed $ p $ with the actual values to evaluate the accuracy.

For a deeper understanding of the PageRank algorithm, you can watch this video: [PageRank Algorithm](https://www.youtube.com/watch?v=TU0ankRcHmo).


In [13]:
import numpy as np  

global myProbs_g  # Initialize global probability variable.  

def gram_schmidt_qr(A):  
    """  
    Perform QR decomposition through the Gram-Schmidt method.  
  
    Parameters:  
    - A (numpy.ndarray): 2D array (m x n) with m rows and n columns of full rank.  

    Returns:  
    - Q (numpy.ndarray): An m x n matrix with orthogonal columns.  
    - R (numpy.ndarray): An n x n upper triangular matrix.  
    """  
    num_rows, num_cols = A.shape  # Retrieve matrix dimensions  
    Q = np.zeros((num_rows, num_cols))  # Allocate memory for Q matrix  
    R = np.zeros((num_cols, num_cols))  # Allocate memory for R matrix  

    for j in range(num_cols):  
        v = A[:, j]  

        for i in range(j):  
            R[i, j] = np.dot(Q[:, i], v)  # Store the projection in R  
            v -= R[i, j] * Q[:, i]  # Update v to remove the projection  

        R[j, j] = np.linalg.norm(v)  # Store the norm in R  
        if R[j, j] == 0:  # Check for zero norm  
            raise ValueError("Zero norm encountered; cannot normalize.")  
        Q[:, j] = v / R[j, j]  # Normalize v and store it in Q  

    return Q, R  # Provide the orthogonal and upper triangular matrices as output  

def compute_null_space_qr(A, tol=1e-15):   
    global myProbs_g  # Access the global variable for probabilities  

    Q, R = gram_schmidt_qr(A.T)  # Perform QR on the transpose of A  

    null_space_indices = np.flatnonzero(np.abs(np.diag(R)) < tol)  # Identify null space indices  
    if null_space_indices.size == 0:  
        raise ValueError("No null space found; please review the tolerance setting.")  

    null_space_vectors = Q[:, null_space_indices]  # Extract the null space vectors  
    norms = np.linalg.norm(null_space_vectors, axis=0)  # Compute the norms of the vectors  
    max_norm_idx = np.argmax(norms)  # Locate the index of the largest norm  

    myProbs_g = null_space_vectors[:, max_norm_idx]  # Select the vector with the largest norm  

    total = np.sum(myProbs_g)  
    if total == 0:  
        raise ValueError("Cannot normalize: sum of probabilities is zero.")  
    
    myProbs_g /= total  # Normalize to create a probability distribution  

    # Visualize the results  
    visualize_graph(graph, testMine=True, myProbs=myProbs_g)  
    visualize_graph(graph)  

    return myProbs_g  # Return the normalized probabilities  

# Execute the function and print the outputs  
null_space = compute_null_space_qr(normalizedTransitions - np.eye(adj.shape[0]))  
print(null_space)  # Display the null space  
print("Stored myProbs for later use:", myProbs_g)  # Output the stored probabilities  
print('Actual Probs:', goalProbs)

[ 0.36363636  0.07828283 -0.05050505  0.12121212  0.24494949 -0.22222222
  0.06060606  0.2020202  -0.08080808  0.28282828]
Stored myProbs for later use: [ 0.36363636  0.07828283 -0.05050505  0.12121212  0.24494949 -0.22222222
  0.06060606  0.2020202  -0.08080808  0.28282828]
Actual Probs: {0: 0.049600466709609034, 8: 0.12211804138958529, 1: 0.04841626473069932, 5: 0.1965643871517563, 2: 0.080704727070993, 9: 0.18993354337728946, 3: 0.09399337594284683, 6: 0.05494773511591966, 4: 0.04841626473069932, 7: 0.11530519378060204}


# Comparing actual and calculated p

In [14]:

goalProbs = np.array(list(goalProbs.values()))
# Mean Squared Error (MSE)
mse = np.mean((goalProbs - myProbs_g) ** 2)
print(f"MSE: {mse}")

MSE: 0.038036012014000906
